# Create Pickle Model
1. Create Pickle 
2. Test Pickle
3. Create Prediction Function
4. Create Visualization Function
5. Test Prediction and Visualization Function

In [1]:
import pandas as pd
import numpy as np
from fbprophet import Prophet

/Users/jisha/.pyenv/versions/3.8.6/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## 1. Create Pickle

In [26]:
population_melt = pd.read_csv('https://raw.githubusercontent.com/jiobu1/labspt15-cityspire-g-ds/main/notebooks/model/population2010-2019/csv/population_melt.csv')
population = pd.read_csv('https://raw.githubusercontent.com/jiobu1/labspt15-cityspire-g-ds/main/notebooks/model/population2010-2019/csv/population_cleaned.csv')

In [3]:
import pickle

In [4]:
m = Prophet(interval_width=.95)

with open('pickle/population_model.pkl', 'wb') as f:
    pickle.dump(m, f)

## 2. Test Pickle

In [5]:
model = pickle.load(open('pickle/population_model.pkl', 'rb'))

## 3. Create Prediction Function

In [27]:
cities_list = list(population['City,State'])

In [28]:
def rnd_series(city):
    subset = population_melt[population_melt['City,State']== city]
    dates = (pd.DataFrame({'ds': pd.to_datetime(population_melt['ds'])}))
    
    return subset

In [30]:
series = [rnd_series(city) for city in cities_list]
res = dict(zip(cities_list, series))

In [51]:
def create_forecast(city, periods):
    population_melt = pd.read_csv('https://raw.githubusercontent.com/jiobu1/labspt15-cityspire-g-ds/main/notebooks/model/population2010-2019/csv/population_melt.csv')
    population_melt.reset_index(level=0, inplace=True)
    df_  = population_melt.loc[population_melt['City,State'] == city][['ds', 'y']]
    df_['ds'] = pd.to_datetime(df_['ds'])
    model = Prophet(interval_width=0.95)
    model.fit(df_)
    forecast = model.make_future_dataframe(periods=periods, freq='Y')
    forecast = model.predict(forecast)
    forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    forecast['ds'] = pd.DatetimeIndex(forecast['ds']).year
    forecast[['yhat', 'yhat_lower', 'yhat_upper']] = forecast[['yhat', 'yhat_lower', 'yhat_upper']].round()#.astype(float)
    # forecast[['yhat', 'yhat_lower', 'yhat_upper']] = forecast[['yhat', 'yhat_lower', 'yhat_upper']].apply(np.ceil)
    return forecast

In [52]:
create_forecast('Akron, OH', 10)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 7.


MemoryError: Unable to allocate 1.70 EiB for an array with shape (245347200327154464,) and data type float64

## 4. Create Visualization Function

In [ ]:
def get_plot(city, periods):
  population_melt[city]['year']= population_melt[city]['ds'].dt.year
  ax = population_melt[city].plot(x = 'ds', y = 'y', label='Observed', figsize= (10, 8))
  df = get_forecast(city, periods)[1:]
  df[['year', 'yhat']].plot(ax = ax, x = 'ds', y = 'yhat', label = "Forecast")
  ax.fill_between(df['year'],
                df['yhat_lower'],
                df['yhat_upper'],
                color='k', 
                alpha=.25)

  ax.set_xlabel('Year')
  ax.set_ylabel('Population')
  plt.title(f"{cityname} Population" )
  plt.legend()

  plt.show()


In [42]:
model = pickle.load(open('pickle/model_df.pkl', 'rb'))

EOFError: Ran out of input

In [ ]:
model.head()